<a href="https://colab.research.google.com/github/MeghanaR123/Video_dehazing_project/blob/main/flask_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import os
import torch
import os
from PIL import Image
from torchvision import transforms as tfs
import glob
from tqdm import tqdm

In [ ]:
from torch import nn
# Define the FFA model (Directly included here)
def default_conv(in_channels, out_channels, kernel_size, bias=True):
    return nn.Conv2d(in_channels, out_channels, kernel_size, padding=(kernel_size // 2), bias=bias)

class PALayer(nn.Module):
    def __init__(self, channel):
        super(PALayer, self).__init__()
        self.pa = nn.Sequential(
            nn.Conv2d(channel, channel // 8, 1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel // 8, 1, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.pa(x)
        return x * y

class CALayer(nn.Module):
    def __init__(self, channel):
        super(CALayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.ca = nn.Sequential(
            nn.Conv2d(channel, channel // 8, 1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel // 8, channel, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.ca(y)
        return x * y

class Block(nn.Module):
    def __init__(self, conv, dim, kernel_size):
        super(Block, self).__init__()
        self.conv1 = conv(dim, dim, kernel_size, bias=True)
        self.act1 = nn.ReLU(inplace=True)
        self.conv2 = conv(dim, dim, kernel_size, bias=True)
        self.calayer = CALayer(dim)
        self.palayer = PALayer(dim)

    def forward(self, x):
        res = self.act1(self.conv1(x))
        res = res + x
        res = self.conv2(res)
        res = self.calayer(res)
        res = self.palayer(res)
        res += x
        return res

class Group(nn.Module):
    def __init__(self, conv, dim, kernel_size, blocks):
        super(Group, self).__init__()
        modules = [Block(conv, dim, kernel_size) for _ in range(blocks)]
        modules.append(conv(dim, dim, kernel_size))
        self.gp = nn.Sequential(*modules)

    def forward(self, x):
        res = self.gp(x)
        res += x
        return res

class FFA(nn.Module):
    def __init__(self, gps, blocks, conv=default_conv):
        super(FFA, self).__init__()
        self.gps = gps
        self.dim = 64
        kernel_size = 3
        pre_process = [conv(3, self.dim, kernel_size)]
        assert self.gps == 3
        self.g1 = Group(conv, self.dim, kernel_size, blocks=blocks)
        self.g2 = Group(conv, self.dim, kernel_size, blocks=blocks)
        self.g3 = Group(conv, self.dim, kernel_size, blocks=blocks)
        self.ca = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(self.dim * self.gps, self.dim // 16, 1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.dim // 16, self.dim * self.gps, 1, padding=0, bias=True),
            nn.Sigmoid()
        )
        self.palayer = PALayer(self.dim)
        post_process = [conv(self.dim, self.dim, kernel_size), conv(self.dim, 3, kernel_size)]
        self.pre = nn.Sequential(*pre_process)
        self.post = nn.Sequential(*post_process)

    def forward(self, x1):
        x = self.pre(x1)
        res1 = self.g1(x)
        res2 = self.g2(res1)
        res3 = self.g3(res2)
        w = self.ca(torch.cat([res1, res2, res3], dim=1))
        w = w.view(-1, self.gps, self.dim)[:, :, :, None, None]
        out = w[:, 0, :] * res1 + w[:, 1, :] * res2 + w[:, 2, :] * res3
        out = self.palayer(out)
        x = self.post(out)
        return x + x1

# Configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gps = 3
blocks = 19

# Load pre-trained model
pretrained_model_dir = '/content/drive/MyDrive/synthetic-objective-testing-set-sots-reside/ffanet-pretrained-weights/' + f'ots_train_ffa_{gps}_{blocks}.pk'  # Update with your model path
net = FFA(gps=gps, blocks=blocks)
net = torch.nn.DataParallel(net)
net.load_state_dict(torch.load(pretrained_model_dir, map_location=device)['model'])
net.to(device)
net.eval()

print("Model loaded successfully!")

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy.core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([scalar])` or the `torch.serialization.safe_globals([scalar])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weigh

In [ ]:
def reduce_video_resolution(input_path, output_path):
    # Open the input video
    cap = cv2.VideoCapture(input_path)

    # Get original width and height of the video
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calculate half resolution
    new_width = original_width // 2
    new_height = original_height // 2

    # Get the frame rate of the original video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Set up output video writer with half resolution and original fps
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print('break')
            break

        # Resize frame to half of original width and height
        resized_frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

        # Write the resized frame to the output video
        out.write(resized_frame)

    # Release resources
    cap.release()
    out.release()
    print("Video saved with half resolution at:", output_path)

In [ ]:
def extract_frames(video_path, output_folder, frame_limit=60):
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)

    # Load the video
    video_capture = cv2.VideoCapture(video_path)

    # Get the total frame count
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    total_frames = min(total_frames, frame_limit)  # Limit to the frame_limit

    # Frame extraction loop
    frame_number = 0
    for i in tqdm(range(total_frames), desc="Extracting Frames"):
        success, frame = video_capture.read()
        if not success or frame_number >= frame_limit:
            break

        # Save each frame as an image
        frame_filename = os.path.join(output_folder, f"frame_{frame_number:04d}.png")
        cv2.imwrite(frame_filename, frame)
        frame_number += 1

    # Release the video capture object
    video_capture.release()
    print(f"Extraction complete. Frames saved to: {output_folder}")

In [ ]:
# Set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Set model parameters
gps = 3
blocks = 19

# Load pretrained model (update with your model path)
pretrained_model_dir =  '/content/drive/MyDrive/synthetic-objective-testing-set-sots-reside/ffanet-pretrained-weights/' + f'ots_train_ffa_{gps}_{blocks}.pk'

# Define FFA model (assuming the FFA class is already defined)
net = FFA(gps=gps, blocks=blocks)
net = nn.DataParallel(net)
net.load_state_dict(torch.load(pretrained_model_dir, map_location=device)['model'])
net.to(device)
net.eval()

# Folder paths
input_frames_dir = '/content/drive/MyDrive/frames_folder/wood'  # Your input frames folder
output_frames_dir = '/content/drive/MyDrive/dehazed_frames_folder/wood'  # Output folder for dehazed frames
os.makedirs(output_frames_dir, exist_ok=True)

# Set frame limit to 60
frame_limit = 60

# Process each frame up to the limit
frame_count = 0
for frame_file in tqdm(sorted(os.listdir(input_frames_dir)), desc="Dehazing Frames"):
    if frame_count >= frame_limit:
        break

    frame_path = os.path.join(input_frames_dir, frame_file)
    hazy_image = Image.open(frame_path)

    # Preprocess the frame
    transform = tfs.Compose([
        tfs.ToTensor(),
        tfs.Normalize(mean=[0.64, 0.6, 0.58], std=[0.14, 0.15, 0.152])
    ])
    hazy_tensor = transform(hazy_image).unsqueeze(0).to(device)

    # Dehaze the frame
    with torch.no_grad():
        dehazed_tensor = net(hazy_tensor)

    # Convert tensor to image and save
    output_image = dehazed_tensor.squeeze(0).cpu().clamp(0, 1)
    output_image = tfs.ToPILImage()(output_image)
    output_image.save(os.path.join(output_frames_dir, frame_file))

    frame_count += 1

print(f"Dehazing complete. Frames saved to: {output_frames_dir}")

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy.core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([scalar])` or the `torch.serialization.safe_globals([scalar])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weigh

In [ ]:
def apply_ahe_to_folder(input_folder, output_folder, clip_limit=4.0, tile_grid_size=(8, 8)):
    # Get a list of all images in the input folder
    image_files = glob.glob(os.path.join(input_folder, "*.*"))

    # Process each image with tqdm progress bar
    for image_path in tqdm(image_files, desc="Processing images"):
        # Read the image
        img = cv2.imread(image_path)

        if img is None:
            print(f"Could not read image {image_path}. Skipping.")
            continue

        # Convert the image to LAB color space
        lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

        # Split the LAB image to L, A, and B channels
        l_channel, a_channel, b_channel = cv2.split(lab_img)

        # Apply CLAHE to the L channel
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
        cl = clahe.apply(l_channel)

        # Merge the CLAHE enhanced L channel with A and B channels
        lab_img = cv2.merge((cl, a_channel, b_channel))

        # Convert the LAB image back to BGR color space
        enhanced_img = cv2.cvtColor(lab_img, cv2.COLOR_LAB2BGR)

        # Define the output path
        output_path = os.path.join(output_folder, os.path.basename(image_path))

        # Save the enhanced image to the output folder
        cv2.imwrite(output_path, enhanced_img)

In [ ]:
def create_video_from_frames(dehazed_frames_dir, output_video_path, frame_limit=60, frame_rate=30):

    # Get the frames, limited by frame_limit
    frame_files = sorted(os.listdir(dehazed_frames_dir))[:frame_limit]

    # Ensure there are frames to process
    if not frame_files:
        print("No frames found in the specified directory.")
        return

    # Read the first frame to get the frame size
    first_frame_path = os.path.join(dehazed_frames_dir, frame_files[0])
    frame = cv2.imread(first_frame_path)
    height, width, layers = frame.shape

    # Define video codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    # Write frames to the video
    for frame_file in tqdm(frame_files, desc="Creating Video"):
        frame_path = os.path.join(dehazed_frames_dir, frame_file)
        frame = cv2.imread(frame_path)
        video_writer.write(frame)

    # Release the VideoWriter
    video_writer.release()
    print(f"Video creation complete. Dehazed video saved to: {output_video_path}")

In [ ]:
!pip install Flask==3.0.0 pyngrok==7.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [ ]:
# ngrok_key = "2seRj7lTIywg503jYkqI9QIkQDs_4XjJECzPYZQw4hnRBrMGo"
ngrok_key = "2qQQHtVmnB7cdMNktHEK0aFXuFd_aVWRYK9Qad895efrAN36"
port = 5000

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://7ece-34-125-102-191.ngrok-free.app'

In [ ]:
from flask import Flask, request, render_template
import os
import shutil
app = Flask(__name__, template_folder='/content/drive/MyDrive/code/templates',
                      static_folder='/content/drive/MyDrive/code/static' )
@app.route('/')
def main():
    return render_template("login.html")

@app.route('/login', methods = ['POST'])
def login():
  if request.method == 'POST':
    un = request.form.get('email')
    psd = request.form.get('pass')
    if(un=="admin@gmail.com" and psd=="admin"):
      return render_template("vindex.html")
    else:
      return render_template("login.html")
  else:
    return render_template("login.html")

@app.route('/success', methods = ['POST'])
def success():
    if request.method == 'POST':
        f = request.files['file']
        upload_folder = os.path.join(app.static_folder, '')
        if not os.path.exists(upload_folder):
            os.makedirs(upload_folder)
        # Save the uploaded file
        fname = os.path.join(upload_folder, f.filename)
        f.save(upload_folder+'input_video.mp4')
        print(upload_folder+'input_video.mp4')

        # Path to the directory you want to manage
        directory_path = '/content/drive/MyDrive/AHE/wood'

        # Check if the directory exists
        if os.path.exists(directory_path):
            shutil.rmtree(directory_path)

        # Create the new directory
        os.makedirs(directory_path)

        #===============write code=========#
        output_video = '/content/drive/MyDrive/input_videos/wood.mp4'  # Path to save resized video
        reduce_video_resolution(upload_folder+'input_video.mp4', output_video)

        if os.path.exists('/content/drive/MyDrive/frames_folder/wood'): shutil.rmtree('/content/drive/MyDrive/frames_folder/wood')

        #call extract_frames
        extract_frames('/content/drive/MyDrive/input_videos/wood.mp4', '/content/drive/MyDrive/frames_folder/wood')

        #call dehazing
        # Set device to GPU if available
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

        # Set model parameters
        gps = 3
        blocks = 19

        # Load pretrained model (update with your model path)
        pretrained_model_dir =  '/content/drive/MyDrive/synthetic-objective-testing-set-sots-reside/ffanet-pretrained-weights/' + f'ots_train_ffa_{gps}_{blocks}.pk'

        # Define FFA model (assuming the FFA class is already defined)
        net = FFA(gps=gps, blocks=blocks)
        net = nn.DataParallel(net)
        net.load_state_dict(torch.load(pretrained_model_dir, map_location=device)['model'])
        net.to(device)
        net.eval()

        # Folder paths
        input_frames_dir = '/content/drive/MyDrive/frames_folder/wood'  # Your input frames folder
        output_frames_dir = '/content/drive/MyDrive/dehazed_frames_folder/wood'  # Output folder for dehazed frames
        os.makedirs(output_frames_dir, exist_ok=True)

        # Set frame limit to 60
        frame_limit = 30

        # Process each frame up to the limit
        frame_count = 0
        for frame_file in tqdm(sorted(os.listdir(input_frames_dir)), desc="Dehazing Frames"):
            if frame_count >= frame_limit:
                break

            frame_path = os.path.join(input_frames_dir, frame_file)
            hazy_image = Image.open(frame_path)

            # Preprocess the frame
            transform = tfs.Compose([
                tfs.ToTensor(),
                tfs.Normalize(mean=[0.64, 0.6, 0.58], std=[0.14, 0.15, 0.152])
            ])
            hazy_tensor = transform(hazy_image).unsqueeze(0).to(device)

            # Dehaze the frame
            with torch.no_grad():
                dehazed_tensor = net(hazy_tensor)

            # Convert tensor to image and save
            output_image = dehazed_tensor.squeeze(0).cpu().clamp(0, 1)
            output_image = tfs.ToPILImage()(output_image)
            output_image.save(os.path.join(output_frames_dir, frame_file))

            frame_count += 1

        print(f"Dehazing complete. Frames saved to: {output_frames_dir}")

        #call AHE
        # Set input and output folder paths
        input_folder = "/content/drive/MyDrive/dehazed_frames_folder/wood"
        output_folder = "/content/drive/MyDrive/AHE/wood"

        # Apply AHE to all images in the input folder and save to output folder
        apply_ahe_to_folder(input_folder, output_folder)

        create_video_from_frames('/content/drive/MyDrive/AHE/wood', '/content/drive/MyDrive/code/static/wood_dehazed_video.mp4', frame_limit=200, frame_rate=30)

        if os.path.exists('/content/drive/MyDrive/code/static/input_video_tmp.mp4'): os.remove('/content/drive/MyDrive/code/static/input_video_tmp.mp4')
        if os.path.exists('/content/drive/MyDrive/code/static/wood_dehazed_video_tmp.mp4'): os.remove('/content/drive/MyDrive/code/static/wood_dehazed_video_tmp.mp4')

        !ffmpeg -i /content/drive/MyDrive/code/static/input_video.mp4 -c:v libx264 -c:a aac -strict experimental /content/drive/MyDrive/code/static/input_video_tmp.mp4
        !ffmpeg -i /content/drive/MyDrive/code/static/wood_dehazed_video.mp4 -c:v libx264 -c:a aac -strict experimental /content/drive/MyDrive/code/static/wood_dehazed_video_tmp.mp4

        in_video = 'input_video_tmp.mp4'
        return render_template("vindex.html", input_video = in_video, output_video='wood_dehazed_video_tmp.mp4')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 05:55:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 05:55:45] "GET /static/css/util.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 05:55:45] "GET /static/vendor/animate/animate.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 05:55:45] "GET /static/vendor/jquery/jquery-3.2.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 05:55:45] "GET /static/vendor/select2/select2.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 05:55:45] "GET /static/vendor/css-hamburgers/hamburgers.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 05:55:45] "GET /static/css/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 05:55:45] "GET /static/vendor/bootstrap/cs